This colab notebook calcuates the DoE scores used in section 6 and Appendix B. 

In [ ]:
cd 'Working Directly'

In [ ]:
pip install transformers

In [ ]:
import warnings
import pandas as pd
import numpy as np
np.random.seed(42)

%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')

In [ ]:
from DoE import get_DoE

#Preparing datasets

In [ ]:

  EA_data = pd.read_csv('EA/hs_AsianPrejudice_20kdataset_cleaned_anonymized.tsv',sep='\t',encoding = "ISO-8859-1")


  EA_data = processor.process_all(EA_data,'text')
  EA_data['label'] = EA_data['expert'].apply(lambda x:label_dict[x])
  EA_data = EA_data[['text','label']]

  train_size = 0.8
  dev_size = 0.3

  EA_train_stratified=EA_data.groupby('label').apply(lambda x: x.sample(frac=train_size, random_state= 100))  #stratified sampling 
  EA_train = EA_data.iloc[[item[1] for item in EA_train_stratified.index]]
  EA_test=EA_data.drop(EA_train.index).reset_index(drop=True)

  EA_dev_stratified=EA_test.groupby('label').apply(lambda x: x.sample(frac=dev_size, random_state= 100))
  EA_dev = EA_test.iloc[[item[1] for item in EA_dev_stratified.index]]
  EA_test=EA_test.drop(EA_dev.index).reset_index(drop=True)

  EA_train = EA_train.reset_index(drop=True)
  EA_dev = EA_dev.reset_index(drop=True)

#Test


In [ ]:
 doe = get_DoE(classifier = 'toxicity',desired_class = 1,added_example = 'this is a virus made by chinese communist party')
 doe

0.76211

In [ ]:
doe = get_DoE(classifier = 'toxicity',desired_class = 1,added_example = 'fucking chinese communist party')
doe

0.96782

#DoE scores

In [ ]:
all_DoEs = []
for example in tqdm(EA_train['text'].values.tolist()):
  doe = get_DoE(classifier = 'toxicity',desired_class = 1,added_example = example)
  all_DoEs.append(doe)



100%|██████████| 15999/15999 [6:06:47<00:00,  1.38s/it]


In [ ]:
EA_train['toxic-DoE'] = all_DoEs
EA_train.to_csv('toxic_DoEs.csv')